# Creating a new process - Orographic precipitation

![Henry Mountains](HenryMountains.jpg "Henry Mountains")

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import xsimlab as xs # modeling framework used for FastScape development
import xarray as xr # xarray is a python package to work with labelled multi-dimensional arrays
%load_ext xsimlab.ipython


We have developed an [orographic precipitation model](https://github.com/fastscape-lem/orographic-precipitation) based on [Smith and Barstad (2004)](https://journals.ametsoc.org/view/journals/atsc/61/12/1520-0469_2004_061_1377_altoop_2.0.co_2.xml) that can easily be included in any landscape evolution model.

We will use it to demonstrate how new processes can be constructed and added to FastScape.

Note that the orographic model can be access through conda using ```conda install orographic_precipitation -c conda-forge```

In [3]:
from orographic_precipitation import compute_orographic_precip

We now develop a very simple process called ```Orography``` that simply takes wind direction as an input and produces a precipitation pattern over a given landscape (topography) of known dimensions

In [4]:
from fastscape.processes import RasterGrid2D, SurfaceTopography, FlowAccumulator

@xs.process
class Orography:

    wind_dir = xs.variable(intent='in', description='wind direction (0 is north)')
    precip = xs.foreign(FlowAccumulator, 'runoff', intent='out')
    dx = xs.foreign(RasterGrid2D, 'dx')
    dy = xs.foreign(RasterGrid2D, 'dy')
    h = xs.foreign(SurfaceTopography, 'elevation')

    @xs.runtime()
    def run_step(self):
        lapse_rate = -5.8
        lapse_rate_m = -6.5
        ref_density = 7.4e-3

        param = {
        'latitude': 40,
        'precip_base': 7,                          # uniform precipitation rate
        'wind_speed': 10,
        'wind_dir': self.wind_dir,                   # wind direction (270: west)
        'conv_time': 1000,                    # conversion time
        'fall_time': 1000,                    # fallout time
        'nm': 0.005,                      # moist stability frequency
        'hw': 5000,                       # water vapor scale height
        'cw': ref_density * lapse_rate_m / lapse_rate   # uplift sensitivity
        }

        self.precip = compute_orographic_precip(self.h, self.dx, self.dy, **param)



In [6]:
from fastscape.models import basic_model

basic_model.drainage

<DrainageArea 'drainage' (xsimlab process)>
Variables:
    shape            [in] <--- grid.shape
    cell_area        [in] <--- grid.cell_area
    stack            [in] <--- flow.stack
    nb_receivers     [in] <--- flow.nb_receivers
    receivers        [in] <--- flow.receivers
    weights          [in] <--- flow.weights
    flowacc         [out] ('y', 'x') flow accumulation from up to d...
    runoff          [out] () or ('y', 'x') 
    area            [out] ('y', 'x') drainage area
Simulation stages:
    initialize
    run_step

We improve the basic_model by substiting the computation of drainage area with a FlowAccumulator which transform any precipitation function into a accumulated flow and by adding our Orography model that takes a wind direction to produce a precipitation function.

In [7]:
spl_model = basic_model.update_processes({'drainage': FlowAccumulator, 'orography': Orography})


Let's explore this new model

In [8]:
spl_model.drainage

<FlowAccumulator 'drainage' (xsimlab process)>
Variables:
    runoff           [in] () or ('y', 'x') surface runoff (source t...
    shape            [in] <--- grid.shape
    cell_area        [in] <--- grid.cell_area
    stack            [in] <--- flow.stack
    nb_receivers     [in] <--- flow.nb_receivers
    receivers        [in] <--- flow.receivers
    weights          [in] <--- flow.weights
    flowacc         [out] ('y', 'x') flow accumulation from up to d...
Simulation stages:
    run_step

We run the model with a wind direction from the South

In [9]:
# %create_setup spl_model --default --verbose
import xsimlab as xs

ds_in = xs.create_setup(
    model=spl_model,
    clocks={'time': np.linspace(0,2e7,201),
           'out': np.linspace(0, 2e7, 21)},
    master_clock = 'time',
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': [101,201],
        # total grid length in (y, x)
        'grid__length': [1e5,2e5],
        # node status at borders
        'boundary__status': 'fixed_value',
        # uplift rate
        'uplift__rate': 1e-3,
        # random seed
        'init_topography__seed': None,
        # diffusivity (transport coefficient)
        'diffusion__diffusivity': 1,
        # bedrock channel incision coefficient
        'spl__k_coef': 1e-5,
        # drainage area exponent
        'spl__area_exp': 0.4,
        # slope exponent
        'spl__slope_exp': 1,
        # wind direction (0 is north)
        'orography__wind_dir': 180,
    },
    output_vars={
        'topography__elevation': 'out',
        'drainage__flowacc': 'out',
        'drainage__runoff': 'out'}
)


In [10]:
with xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(model=spl_model)

             0% | initialize 

In [11]:
from ipyfastscape import TopoViz3d

app = TopoViz3d(ds_out, canvas_height=600, time_dim="out")

app.components['background_color'].set_color('lightgray')
app.components['vertical_exaggeration'].set_factor(5)
app.components['timestepper'].go_to_time(ds_out.out[-1])

app.show()

Output(layout=Layout(height='640px'))